# Modifications

Link to Interactive Notebook: 
https://colab.research.google.com/drive/1sbJTsgCsAQwCkGdLXK7EdgkaTpiTGBM1#scrollTo=T71qxHyh9p23

1. Reduced Input Dimension
2. Different handling of xco2: merge the value in as an additional input in the fully connected layer (?)

In [ ]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split

#!module load cuda11.0/toolkit cuda11.0/blas cudnn8.0-cuda11.0

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#tf.config.list_physical_devices()

sys.path.insert(0, '../src')

from utils import df_to_xarray,read_xarray,plot_image,preprocess_image_reduced

In [ ]:
# Reading Data
dir_name="../data/data1"
val_dir_name="../data/data2"


chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)



In [ ]:
def convert_nan(arr):
    nans=np.isnan(arr)
    min_val=arr[~nans].min()
    #print(min_val)
    arr[nans]=min_val-1
    return arr

def add_dimension(arr):
    images=np.expand_dims(arr, axis=3)
    return images

def scale_image(arr):
    ## Normal
    #arr=(arr-np.mean(arr))/np.std(arr)
    
    ## Min-Max
    # min_val=arr.min()
    # max_val=arr.max()
    # arr=arr/(min_val-max_val)

    ## Image Scale
    min_pixel = arr.min() 
    max_pixel = arr.max()
    new_min = 0
    new_max = 255
    arr = (arr-min_pixel)*(255)/(max_pixel-min_pixel)+new_min 
    return arr
  
def preprocess_image(data,train=False):
    if train:
      return add_dimension(convert_nan(data))
    else:
      return add_dimension(scale_image(convert_nan(data))/255.0)

def preprocess_image_reduced(data,xco2=False):
  """
  dimension reduced the output should be  (180,360,5)
  """
  if xco2:
    return data
  return scale_image(convert_nan(data))

In [ ]:
chl_images=preprocess_image_reduced(chl.Chl.data)
mld_images=preprocess_image_reduced(mld.MLD.data)
sss_images=preprocess_image_reduced(sss.SSS.data)
sst_images=preprocess_image_reduced(sst.SST.data)
xco2_images=preprocess_image_reduced(xco2.XCO2.data,xco2=True)
pco2_images=preprocess_image_reduced(pco2.pCO2.data,pco2=True)

X = np.stack((chl_images, mld_images, sss_images, sst_images,xco2_images), axis = 1)


In [4]:
X=X.reshape((421,180,360,5))
X.shape

(421, 180, 360, 5)

In [5]:
INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=4,activation='relu', padding="SAME")

base_model = keras.models.Sequential([
    DefaultConv2D(filters=64, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.3),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),

    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=64),    
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])

base_model.summary()

In [ ]:
base_model.compile(loss="mean_squared_error", optimizer="nadam", metrics=["mean_squared_error"])

In [ ]:
model_path="../models/base_model/base_model.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(X,pco2_images, epochs=100, validation_data=(X,pco2_images),workers=-1,batch_size=32,callbacks=callbacks)



In [ ]:
predicted_image=base_model.predict(X[419:421],verbose=1)

In [ ]:
plot_image(np.squeeze(predicted_image[1]))

In [ ]:
difference=np.squeeze(pco2_images[419:421][1])-np.squeeze(predicted_image[0])
plot_image(difference)

In [ ]:
plot_image(np.squeeze(pco2_images[419:421][1]))